# Analysis of Census Data

Analysis of the 2013-2017 American Community Survey 5-year Public Use Microdata Sample files of population and housing unit records. 

Introduction and indicator section for website 

Counts, not percentages 


### Configuration

Load the software packages needed. 

In [2]:
import numpy as np 
import pandas as pd
import community_profiles.datasets as cp_data

### Load Census Data

In [3]:
pumas = cp_data.PUMAs.get().sort_values('puma_id')
house = cp_data.houses.get(fresh = True)
persons = cp_data.persons.get(fresh = True) 

### Population

In [4]:
group = persons.groupby('PUMA')
pop = group['person_weight'].sum().reset_index().rename(columns = {'person_weight' : 'population'})
population = pop['population']

In [5]:
pop

,PUMA,population
0,3201,155888
1,3202,147797
2,3203,132229
3,3204,169887
4,3205,166737
5,3206,138494
6,3207,103034
7,3208,136218
8,3209,137460
9,3210,151809


### Population by age and sex

0-9, 10-19, 20-29, ect. 

 MAYBE BETTER WAY TO DO THIS? 

In [6]:
bins =  np.array([0,10,20,30,40,50, 60, 70, 80,100])  

# male 
male = persons.loc[persons['SEX'] == 'male']
group_m = male.groupby([pd.cut(male.AGEP, bins, right = False), 'PUMA'])
male = group_m['person_weight'].sum().reset_index()
male = male.pivot_table(values='person_weight', 
                                index='PUMA', 
                                columns=['AGEP']).reset_index(drop = True).add_prefix('male_')
male = male.div(population, axis=0) *100

# female 
female = persons.loc[persons['SEX'] == 'female']
group_f = female.groupby([pd.cut(female.AGEP, bins, right = False), 'PUMA'])
female = group_f['person_weight'].sum().reset_index()
female = female.pivot_table(values='person_weight', 
                            index='PUMA', 
                            columns=['AGEP']).reset_index(drop = True).add_prefix('female_') 
female = female.div(population, axis=0) *100

#### Age under 18 

In [7]:
under18 = persons.loc[persons['AGEP'] < 18 ]
groups = under18.groupby([ 'PUMA'])
under18 = groups['person_weight'].sum().reset_index(drop = True)
under18 = under18 / population *100

#### Age 65 and older

In [8]:
plus65 = persons.loc[persons['AGEP'] >= 65 ]
groups1 = plus65.groupby(['PUMA'])
plus65 = groups1['person_weight'].sum().reset_index(drop = True) 
plus65 = plus65  / population *100

### Area and Density 



In [9]:
area = ((pumas['geometry'].area) * 3.58701e-8).reset_index(drop = True)
density = population / area 

### Foreign-born 


In [10]:
foreign = persons.loc[persons['NATIVITY']== 'foreign']
group = foreign.groupby(['PUMA'])
foreign_born = group['person_weight'].sum().reset_index(drop = True)
foreign_born =  foreign_born / population *100

### Race and Hispanic origin

#### Race


sum other

In [11]:
group = persons.groupby(['RACE', 'PUMA'])
race = group['person_weight'].sum().reset_index()

race = race.pivot(index='PUMA', 
                  columns='RACE', 
                  values = 'person_weight').reset_index(drop =True)
race.head()

RACE,american indian alone,american indian and alaska native tribes specified,asian alone,black alone,native hawaiian/other pacific alone,some other race alone,two or more races,white alone
0,140.0,217.0,12846.0,15781.0,136.0,4107.0,3343.0,119318.0
1,402.0,20.0,19974.0,41824.0,16.0,14392.0,4245.0,66924.0
2,442.0,133.0,5411.0,37711.0,178.0,12485.0,4055.0,71814.0
3,56.0,529.0,11389.0,124186.0,25.0,11316.0,4793.0,17593.0
4,826.0,408.0,4832.0,37374.0,125.0,34216.0,6499.0,82457.0


In [12]:
other = race.iloc[:, [0,1,4,5,6]].sum(axis =1)
race['other'] = other 

drop summed columns 

In [13]:
race = (race.drop(race.columns[[0,1,4,5,6]], axis =1 ))


#### Hispanic 

1. Not Spanish/Hispanic/Latino

    All other # Hispanic

In [14]:
hisp =  persons.loc[persons['HISP'] != 1] 
group = hisp.groupby(['HISP', 'PUMA'])
hisp = group['person_weight'].sum().reset_index()
hisp = hisp.pivot(index='PUMA', 
                  columns='HISP', 
                  values = 'person_weight').reset_index(drop =True)
hisp = hisp.iloc[:, :].sum(axis =1)

In [15]:
race['hispanic'] = hisp 

In [16]:
race = race.div(population, axis=0) *100

#### Race and Hispanic combined in one dataframe

In [17]:
race.head()

RACE,asian alone,black alone,white alone,other,hispanic
0,8.240532,10.123294,76.540850,5.095325,7.424561
1,13.514483,28.298274,45.281027,12.906216,19.769007
2,4.092143,28.519462,54.310325,13.078069,22.340788
3,6.703868,73.099178,10.355707,9.841247,14.776292
4,2.897977,22.414941,49.453331,25.233751,49.138464


### Educational attainment ( Age 25 + with a least a bachelor's)  

SCHL

21 .Bachelor's degree

22 .Master's degree

23 .Professional degree beyond a bachelor's degree

24 .Doctorate degree


In [18]:
bach_plus =  persons.loc[(persons['SCHL'] >= 21) & (persons['AGEP'] >= 25)] 
groups = bach_plus.groupby(['PUMA'])
edu = groups['person_weight'].sum().reset_index(drop = True)
edu =  edu / population *100

### Limited English 

1 .Very well

2 .Well

3 .Not well

4 .Not at all

Limited( 3 + 4 ) 

In [19]:
lim_eng = persons.loc[persons['ENG'] >= 3]
group = lim_eng.groupby(['PUMA'])
lim_eng = group['person_weight'].sum().reset_index(drop = True)
lim_eng =  lim_eng / population *100

### Unemployed

3. unemployed

In [20]:
workers_group = persons.groupby(['EMPLOY', 'PUMA'])
workers = workers_group['person_weight'].sum().reset_index() 
workers = workers.pivot(index='PUMA', columns='EMPLOY', 
                      values = 'person_weight').reset_index(drop=True)


In [21]:
unemploy = workers[3] / population *100 

In [22]:
workers = workers.iloc[:, [0,3]].sum(axis =1)

### Commuting

#### Commute time

In [23]:
group = persons.groupby(['PUMA'])
mean_commute = group['COM_TIME'].mean().reset_index(drop=True)

#### Commute type


In [24]:
group = persons.groupby(['COM_TYP', 'PUMA'])
commute_t = group['person_weight'].sum().reset_index()
commute_t = commute_t.pivot(index='PUMA', 
                            columns='COM_TYP',  
                            values = 'person_weight').reset_index(drop=True)

Public 

In [25]:
public = commute_t.iloc[:, [1,3,6,7,8]].sum(axis =1)
commute_t['public'] = public

Private

In [26]:
private = commute_t.iloc[:, [2,4,9]].sum(axis =1)
commute_t['private'] = private

Drop summed columns dataframe

In [27]:
commute_t = commute_t.drop(commute_t.columns[[1,2,3,4,6,7,8,9]], axis =1 )
commute_t = commute_t.div(workers, axis=0) *100

### Number of houses 

In [28]:
group = house.groupby('PUMA')
house_count = group['house_weight'].sum().reset_index(drop=True)

### Household Income 

Median 

In [29]:
# says use ADJINC to adjust HINCP to constant $ 
# Don't use and get this result IDK 
group = house.groupby(['PUMA'])
med_income = (group['house_income']).median().reset_index(drop=True)

### Rent Burden (35% or more of income goes to rent)

not sure if this is right 


In [30]:
house['rent_income'] = (house['month_rent'] * 12 / house['house_income']) *100 
above35 = house.where(house['rent_income'] >= 35) 
group = above35.groupby(['PUMA'])
rent_burden = group['house_weight'].sum().reset_index(drop=True)
rent_burden_percent = (rent_burden / house_count) * 100

### Combine into one dataframe

In [35]:
d = {'area' : area,
     'population_density' : density,
     'under_18' : under18,
     '65_plus' : plus65,
     'foreign_born' : foreign_born,
     'educational_attainment' : edu,
     'limited_english' : lim_eng,
     'unemployed' : unemploy,
     'median_income' : med_income,
     'rent_burden' : rent_burden_percent,
     'commute_time' : mean_commute,
      }
df = pd.DataFrame(d)

In [36]:
df1 = pd.concat([ pop, df,  commute_t, race, male, female], axis = 1)

In [37]:
df1

,PUMA,population,area,population_density,under_18,65_plus,foreign_born,educational_attainment,limited_english,unemployed,...,"male_[80, 100)","female_[0, 10)","female_[10, 20)","female_[20, 30)","female_[30, 40)","female_[40, 50)","female_[50, 60)","female_[60, 70)","female_[70, 80)","female_[80, 100)"
0,3201,155888,27.408271,5687.626230,19.760341,19.592271,20.093272,18.502386,7.076234,3.174074,...,2.234296,5.503952,5.104947,5.736811,6.771528,6.044724,6.785641,6.804886,4.552628,3.636585
1,3202,147797,9.130013,16188.037485,26.930858,11.988065,29.135233,12.345311,13.007030,6.128676,...,1.150903,8.157811,6.246406,7.197710,7.094867,7.349946,6.383079,4.878989,2.293010,2.633342
2,3203,132229,10.872806,12161.442177,26.727117,10.036376,11.296312,9.429853,5.646265,5.643240,...,1.003562,7.753216,6.260351,7.281307,8.323439,6.999221,6.624114,4.334148,2.618185,1.604792
3,3204,169887,8.833050,19233.107103,24.117207,12.985102,13.663788,9.535750,6.506678,6.954623,...,0.743435,7.006422,7.072348,8.343193,7.301912,6.423093,7.161819,6.054613,3.346931,1.798843
4,3205,166737,11.489575,14512.025167,28.164115,8.574582,9.492794,9.165332,9.705104,6.128814,...,0.499589,7.263535,7.620984,8.327486,8.175150,6.158801,6.119817,4.240810,2.244253,0.982985
5,3206,138494,19.518036,7095.693597,18.109810,15.567461,5.481826,30.285067,0.734328,3.952518,...,1.643392,5.471717,4.539547,9.665401,7.849438,5.429838,7.226306,6.630612,3.400147,2.918538
6,3207,103034,6.832156,15080.744441,21.709339,11.868898,3.661898,7.891570,1.752819,5.594270,...,1.179222,6.107693,7.653784,10.062698,6.574529,5.826232,7.373294,5.228371,2.880603,2.318652
7,3208,136218,11.983238,11367.378597,23.286937,12.970386,8.029042,13.468851,1.046851,4.855452,...,1.074014,6.190078,7.947555,10.309210,6.001410,5.937541,6.675329,5.944882,3.298389,2.795519
8,3209,137460,6.629009,20736.129826,10.755856,13.545759,14.114652,58.402444,3.115088,2.721519,...,1.083224,4.001891,1.862360,15.421941,11.321112,5.034192,4.826131,4.600611,3.028517,2.215917
9,3210,151809,17.405158,8722.069645,21.044207,10.886048,13.294337,14.278468,2.459011,6.020723,...,0.948560,5.813226,7.310502,12.258825,7.294034,5.430508,6.083961,4.560336,2.877300,2.038746
